In [ ]:
from pprint import pprint
import ee

In [ ]:
# Define Earth Engine Constants
aoi: ee.Geometry = ee.FeatureCollection("<ASSET-ID>").geometry()
eetable = ee.FeatureCollection("<ASSET-ID>")


# Load the input Data Set

In [ ]:
# Load in the nova scotia data set
from cnwi import datasets

ns_data_set = datasets.NovaScotia()
pprint(ns_data_set.__dict__)

# Build Stack for Building of Samples

In [ ]:
from cnwi import sar, datacube

# Sentinel -1 
s1_col = ee.ImageCollection(ns_data_set.sentienl1).filterBounds(aoi)
sar_inpts = sar.build_s1_inputs(s1_col)
alos_inpts = sar.build_alos_inpts(2019).clip(aoi)

# Data Cube
dc_col = ee.ImageCollection(ns_data_set.data_cube).filterBounds(aoi)
dcpp = datacube.prep_data_cube(dc_col)
dc_inpts = datacube.build_data_cube_inpts(dcpp)
dc_inpts.bandNames()

# Terrain Analysis
elev_inpt = ee.ImageCollection(ns_data_set.terrain_analysis).filterBounds(aoi).mosaic()
elev_inpt.bandNames()

# create the stack
stack = ee.Image.cat(sar_inpts, alos_inpts, dc_inpts, elev_inpt)

# Prep Training Data and Generate Samples

In [ ]:
# Sample the stack
from cnwi import trainingd as td

preped = td.prep_training_data(col=eetable, class_property='land_cover')

samples = td.generate_samples(col=preped, stack=stack)
training, validation = td.partition_training(samples, 0.7)

# Build, Train and Classify

In [ ]:
from cnwi import rf

clf = rf.RandomForestModel(
    numberOfTrees=1000
)

trained = clf.train(
    training_data=training,
    predictors=stack.bandNames(),
    classProperty='value'
)

clf_img = stack.classify(trained)

# Accuracy Assessments

In [ ]:
from cnwi import acas

indpnt = acas.independent(
    sample = validation,
    model = trained,
    class_property='value',
    label='land_cover'
)

# Export Metrics to Storage

In [ ]:
# export metrics to cloud or drive
task = ee.batch.Export.table.toDrive(
    collection=indpnt,
    description='NS-Example-Validation-2',
    folder='NS-Example-Validation',
    fileFormat='GeoJSON'
).start()


In [ ]:
# build metics csvs